In [ ]:
# Hacky way to schedule. Here I'm setting these to sleep until the gpus should be free.
# At the end of the notebooks  os._exit(00) will kill the kernel freeing the gpu. 
#                          Hours to wait
# import time; time.sleep( 6 * (60*60))

# G only ACGT Probabilities Hilbert Conv1D

> 

In [ ]:
import os, json, re

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna  import *
from EnvDL.dlfn import * # includes LSUV_

from tqdm import tqdm

# import os
# import numpy as np
# import pandas as pd

# import plotly.express as px
# import plotly.io as pio
# pio.templates.default = "plotly_white"

# import hilbertcurve
# from hilbertcurve.hilbertcurve import HilbertCurve

# from EnvDL.core import * # includes remove_matching_files
# from EnvDL.dna import *
# from EnvDL.dlfn import *

# from tqdm import tqdm

In [ ]:
use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F # F.mse_loss

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger


device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.24_g2fc_G_ACGT_Hilbert_conv2d/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
# phno = pd.read_csv(load_from+'phno.csv')
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx	Geno_Idx	Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')
# GMat = np.load(load_from+'GMat.npy')
# ACGT_OneHot = np.load(load_from+'ACGT_OneHot.npy')
# ACGT = np.load(load_from+'ACGT.npy')
ACGT_hilb = np.load(load_from+'ACGT_hilb.npy')
# SMat = np.load(load_from+'SMat3.npy')
# WMat = np.load(load_from+'WMat3.npy')
# MMat = np.load(load_from+'MMat3.npy')

In [ ]:
# can't reduce size because there are missing values in the hilbert curve.
# reduce Size of ACGT ----
# Since these are probabilities we can immediately save 1/4 of the space needed by using ACG instead of ACGT
# ACGT_hilb = ACGT_hilb[:, 0:-1, :]

In [ ]:
# # dataloader_batch_size = 8 #16 #64
# # run_epochs = 200

# use_gpu_num = 0

# # Imports --------------------------------------------------------------------
# import torch
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# from torch import nn

# device = "cuda" if torch.cuda.is_available() else "cpu"
# if use_gpu_num in [0, 1]: 
#     torch.cuda.set_device(use_gpu_num)
# print(f"Using {device} device")

### Create train/test validate indicies from json

In [ ]:
# rng = np.random.default_rng(9230473) # note, must use rng.shuffle(arr) below for this to take effect.

In [ ]:
# pct_cumsum_thresh = .9

# # make a df to aid in creating train/test splits
# # the plan is to shuffle the rows of the df, calculate the cumulative sum of the percents obs, then 
# # the entries above and below a given percent will be the train/test.
# obs_per_hybrid = phno.assign(n = 1).groupby('Hybrid').count().reset_index()
# obs_per_hybrid = obs_per_hybrid.loc[:, ['Hybrid', 'n']]
# obs_per_hybrid['pct'] = obs_per_hybrid.n / obs_per_hybrid.n.sum()
# obs_per_hybrid['pct_cumsum'] = np.nan
# obs_per_hybrid['random_order'] = 0

# obs_per_hybrid

In [ ]:
# # fill in the random values to sort on
# arr = np.arange(obs_per_hybrid.shape[0])
# rng.shuffle(arr)
# obs_per_hybrid.random_order = arr

# obs_per_hybrid = obs_per_hybrid.sort_values('random_order')
# obs_per_hybrid['pct_cumsum'] = obs_per_hybrid.pct.cumsum()
# obs_per_hybrid

In [ ]:
# # Convert back into phno indices
# train_hybrids = list(obs_per_hybrid.loc[(obs_per_hybrid.pct_cumsum <= pct_cumsum_thresh), 'Hybrid'])
# test_hybrids  = list(obs_per_hybrid.loc[(obs_per_hybrid.pct_cumsum >  pct_cumsum_thresh), 'Hybrid'])

# train_idx = phno.loc[(phno.Hybrid.isin(train_hybrids)), ].index
# test_idx  = phno.loc[(phno.Hybrid.isin(test_hybrids)), ].index

In [ ]:
# [len(train_idx), len(test_idx)]

In [ ]:
# confirm all observation idxs are have genomic information
# assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

In [ ]:
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)

temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)

In [ ]:
train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
# confirm all observation idxs are have genomic information
assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

## One Hot Encoded

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
training_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT_hilb).to(torch.float), 
                idx_original = torch.from_numpy(np.array(train_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

In [ ]:
validation_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT_hilb).to(torch.float), 
                idx_original = torch.from_numpy(np.array(test_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

In [ ]:
next(iter(training_dataloader))[0].shape

torch.Size([50, 4, 256, 512])

In [ ]:
ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [2, 2, 2, 2], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None
    )(xin)

NameError: name 'xin' is not defined

In [ ]:
model = ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [2, 2, 2, 2], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None
    )

max_epoch = 10
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-hilb-res-4rep2-from-pytorch")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: tb_logs/g-acgt-hilb-res-4rep2-from-pytorch
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type     | P

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
model = ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [4 for i in range(4)], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None
    )

max_epoch = 10
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-hilb-res-4rep4-from-pytorch")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs/g-acgt-hilb-res-4rep4-from-pytorch
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type     | Params
----------------------------------
0 | mod  | ResNet2d | 23.7 M
----------------------------------
23.7 M    Trainable params
0         Non-trainable params
23.7 M    Total params
94.886    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
xin = torch.randn((50, 4, 256, 512))

model = nn.Sequential(
    nn.Conv2d(4, 8, kernel_size=3, stride=2, padding=1),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.Conv2d(8, 8, 3, 2, 1),
    nn.BatchNorm2d(8),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
    nn.Linear(8, 1)
)

model(xin).shape


torch.Size([50, 1])

In [ ]:
max_epoch = 10
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-hilb-no-res-8deep-batch-norm")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
# from the Fixup paper's repository 
# https://github.com/hongyi-zhang/Fixup/blob/master/cifar/models/fixup_resnet_cifar.py
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class FixupBasciBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(FixupBasicBlock, self).__init__()
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.bias1a = nn.Parameter(torch.zeros(1))
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bias1b = nn.Parameter(torch.zeros(1))
        self.relu = nn.ReLU(inplace=True)
        self.bias2a = nn.Parameter(torch.zeros(1))
        self.conv2 = conv3x3(planes, planes)
        self.scale = nn.Parameter(torch.ones(1))
        self.bias2b = nn.Parameter(torch.zeros(1))
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x + self.bias1a)
        out = self.relu(out + self.bias1b)

        out = self.conv2(out + self.bias2a)
        out = out * self.scale + self.bias2b

        if self.downsample is not None:
            identity = self.downsample(x + self.bias1a)
            identity = torch.cat((identity, torch.zeros_like(identity)), 1)

        out += identity
        out = self.relu(out)

        return out


class FixupResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(FixupResNet, self).__init__()
        self.num_layers = sum(layers)
        self.inplanes = 16
        # self.conv1 = conv3x3(3, 16)
        self.conv1 = conv3x3(4, 16)
        self.bias1 = nn.Parameter(torch.zeros(1))
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.bias2 = nn.Parameter(torch.zeros(1))
        self.fc = nn.Linear(64, num_classes)

        for m in self.modules():
            if isinstance(m, FixupBasicBlock):
                nn.init.normal_(m.conv1.weight, mean=0, std=np.sqrt(2 / (m.conv1.weight.shape[0] * np.prod(m.conv1.weight.shape[2:]))) * self.num_layers ** (-0.5))
                nn.init.constant_(m.conv2.weight, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.weight, 0)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1:
            downsample = nn.AvgPool2d(1, stride=stride)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x + self.bias1)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x + self.bias2)

        return x



In [ ]:
def fixup_resnet20(**kwargs):
    """Constructs a Fixup-ResNet-20 model.

    """
    model = FixupResNet(FixupBasicBlock, [3, 3, 3], **kwargs)
    return model


In [ ]:
model = fixup_resnet20(num_classes=1).to('cuda')
model(next(iter(training_dataloader))[0]).shape

In [ ]:
# LSUV_(model, next(iter(training_dataloader))[0]) # woah! I was not expecting this to work

In [ ]:
# Module for training subnetworks.
class plDNN_ACGT(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        g_i, y_i = batch
        # pred, out = self.mod(g_i)
        pred = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        g_i, y_i = batch
        # pred, out = self.mod(g_i)
        pred = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
max_epoch = 10
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-hilb-res-f20")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    

#         def Linear_block(in_size, out_size, drop_pr):
#             block = nn.Sequential(
#                 nn.Linear(in_size, out_size),
#                 nn.ReLU(),
#                 nn.Dropout(drop_pr)
#             )
#             return(block)         
        
        
#         def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
#             block = nn.Sequential(
#                 nn.Conv1d(
#                     in_channels= in_channels, # second channel
#                     out_channels= out_channels,
#                     kernel_size= kernel_size,
#                     stride= stride
#                 ), 
#                 nn.MaxPool1d((kernel_size,), stride=stride)
#             )
#             return(block)
        
        self.x_network = nn.Sequential(
            nn.Conv2d(
                    in_channels= 4, 
                    out_channels= 4,
                    kernel_size= (3, 3),
                    stride= 2,
                    padding = 1,
                    bias = True
                ),
            nn.Conv2d(
                    in_channels= 4, 
                    out_channels= 4,
                    kernel_size= (3, 3),
                    stride= 2,
                    padding = 1,
                    bias = True
                ),
            nn.Conv2d(
                    in_channels= 4, 
                    out_channels= 4,
                    kernel_size= (3, 3),
                    stride= 2,
                    padding = 1,
                    bias = True
                ),
            nn.Conv2d(
                    in_channels= 4, 
                    out_channels= 4,
                    kernel_size= (3, 3),
                    stride= 2,
                    padding = 1,
                    bias = True
                ),
            nn.Conv2d(
                    in_channels= 4, 
                    out_channels= 4,
                    kernel_size= (3, 3),
                    stride= 2,
                    padding = 1,
                    bias = True
                )
        )
        
        self.x_pred = nn.Sequential(
            nn.Flatten(),            
            nn.Linear(512, 1)
        )
        
    def forward(self, x):
        out = self.x_network(x)
        pred = self.x_pred(out)
        return pred, out

model = NeuralNetwork().to(device)

# model(next(iter(training_dataloader))[0])[0].shape

# torch.Size([50, 4, 256, 512])

In [ ]:
LSUV_(model, next(iter(training_dataloader))[0])

In [ ]:
# Module for training subnetworks.
class plDNN_ACGT(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
max_epoch = 200
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-hilb-res")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
torch.save(DNNG.mod, cache_path+'g-acgt-hilb'+'.pt')